### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [61]:
# Dependencies and Setup
import pandas as pd
import os
import csv

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Calculate the percentage of students who passed math **and** reading (% Overall Passing)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [62]:
# Calculate the number of unique schools and students in the DataFrame
school_count = school_data["School ID"].count()
student_count = student_data["Student ID"].count()

# Calculate the total school budget
budget_sum = school_data['budget'].sum()

# Calculate the average of math and reading score
avg_math = student_data['math_score'].mean()
avg_read = student_data['reading_score'].mean()

# Calculate the percentage of student that passed math and reading test
pct_math = (100*(student_data[student_data['math_score'] >= 70].count()["student_name"])/student_count)
pct_read = (100*(student_data[student_data['reading_score'] >= 70].count()["student_name"])/student_count)
passed_both = student_data[(student_data['math_score'] >= 70) & (student_data['reading_score'] >= 70)]
pct_overall = (100*(passed_both.count()["student_name"])/student_count)


# Place all of the data found into a summary DataFrame
summary_df = pd.DataFrame({"Total Schools": [school_count], "Total Students": [student_count],  
                           "Total Budget": [budget_sum],
                           "Average Math Score" : [avg_math], "Average Reading Score" : [avg_read],
                           "% Passing Math" : [pct_math], "% Passing Reading" : [pct_read],
                           "% Overall Passing" : [pct_overall]})

# Format budget column and hide index
format_dict = {'Total Students':'{0:,.0f}', 'Total Budget':'${0:,.2f}'}
for key, value in format_dict.items(): 
    summary_df[key] = summary_df[key].apply(value.format)

summary_df

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * % Overall Passing (The percentage of students that passed math **and** reading.)
  
* Create a dataframe to hold the above results

In [66]:
#sort school list alhabetically 
school_df = school_data.sort_values("school_name")
# Add and calculate columns for school budget
school_df['Per Student Budget'] = (school_df['budget'] / school_df['size'])
school_df.head()

# get average score of math and reading for each school
average_df = student_data.groupby('school_name').mean()[['math_score','reading_score']]

# count the number of student that passed math and reading for each school
pct_math = (student_data[student_data['math_score'] >= 70]).groupby('school_name').count()[['math_score']]
pct_read = (student_data[student_data['reading_score'] >= 70]).groupby('school_name').count()[['reading_score']]

# count the number of student that passed reading and math
passed_both = student_data[(student_data['math_score'] >= 70) & (student_data['reading_score'] >= 70)]
pct_both = passed_both.groupby('school_name').count()[['student_name']]

# combine the lists
calc_df = pd.merge(average_df,pct_math, on='school_name')
calc_df = pd.merge(calc_df,pct_read, on='school_name')
calc_df = pd.merge(calc_df,pct_both, on='school_name')
# combine the lists with school data
calc_df = pd.merge(school_df,calc_df, on='school_name')

# Add and calculate columns for average
calc_df['% Passing Math'] = (100* calc_df['math_score_y'] / calc_df['size'])
calc_df['% Passing Reading'] = (100* calc_df['reading_score_y'] / calc_df['size'])
calc_df['% Passing Overall'] = (100* calc_df['student_name'] / calc_df['size'])
# Delete columns for numebr of students that passed math, reading, and both
district_info_df = (calc_df.drop(['School ID','math_score_y', 'reading_score_y','student_name'], axis=1))
district_info_df.head()

# Rename columns to clean up the look
district_info_df = district_info_df.rename(columns={"school_name":"School Name", "type":"School Type",
                     "size":"Total Students","budget":"Total School Budget", 
                     "math_score_x" :"Average Math Score","reading_score_x" :"Average Reading Score"})

# Format budget column and hide index
format_dict = {'Total School Budget':'${0:,.0f}', 'Per Student Budget':'${0:,.0f}'}
district_info_df.style.format(format_dict).hide_index()

School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Bailey High School,District,4976,"$3,124,928",$628,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356",$582,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411",$639,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916",$644,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500",$625,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020",$652,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087",$581,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635",$655,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650",$650,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858",$609,83.839917,84.044699,94.594595,95.945946,90.540541


## Top Performing Schools (By % Overall Passing)

* Sort and display the top five performing schools by % overall passing.

In [4]:
# Displaying Top Performing Schools using nlargest method uof sing the key (% Overall Passing):
format_dict = {'Total School Budget':'${0:,.0f}', 'Per Student Budget':'${0:,.0f}'}
district_info_df.nlargest(5, '% Passing Overall').style.format(format_dict).hide_index()

School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Cabrera High School,Charter,1858,"$1,081,356",$582,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130",$638,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500",$625,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574",$578,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858",$609,83.839917,84.044699,94.594595,95.945946,90.540541


## Bottom Performing Schools (By % Overall Passing)

* Sort and display the five worst-performing schools by % overall passing.

In [5]:
# Displaying Bottom Performing Schools using nsmallest() method uof sing the key (% Overall Passing):
format_dict = {'Total School Budget':'${0:,.2f}', 'Per Student Budget':'${0:,.2f}'}
district_info_df.nsmallest(5, '% Passing Overall').style.format(format_dict).hide_index()

School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [73]:
m_df = (student_data[student_data['grade'] == '9th']).groupby('school_name').mean()[['math_score']]
m_df['10th'] = (student_data[student_data['grade'] == '10th']).groupby('school_name').mean()[['math_score']]
m_df['11th'] = (student_data[student_data['grade'] == '11th']).groupby('school_name').mean()[['math_score']]
m_df['12th'] = (student_data[student_data['grade'] == '12th']).groupby('school_name').mean()[['math_score']]
m_df = m_df.rename(columns={"math_score":"9th"})
m_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


In [6]:
# count the number of student that passed math and reading for each school
math_9gr = (student_data[student_data['grade'] == '9th']).groupby('school_name').mean()[['math_score']]
math_10gr = (student_data[student_data['grade'] == '10th']).groupby('school_name').mean()[['math_score']]
math_11gr = (student_data[student_data['grade'] == '11th']).groupby('school_name').mean()[['math_score']]
math_12gr = (student_data[student_data['grade'] == '12th']).groupby('school_name').mean()[['math_score']]

# combine the lists and rename columns to clean up the look
math_df = pd.merge(math_9gr,math_10gr, on='school_name')
math_df  = math_df .rename(columns={"math_score_x":"9th","math_score_y":"10th"})

math_df = pd.merge(math_df,math_11gr, on='school_name')
math_df = pd.merge(math_df,math_12gr, on='school_name')
math_df  = math_df .rename(columns={"math_score_x":"11th","math_score_y":"12th"})

math_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248


,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [24]:
pd.pivot_table(student_data,index=["school_name"],values=["reading_score"],
               columns=["grade"]).iloc[:,[3,0, 1, 2]]

reading_score                                 
grade                           9th       10th       11th       12th
school_name                                                         
Bailey High School        81.303155  80.907183  80.945643  80.912451
Cabrera High School       83.676136  84.253219  83.788382  84.287958
Figueroa High School      81.198598  81.408912  80.640339  81.384863
Ford High School          80.632653  81.262712  80.403642  80.662338
Griffin High School       83.369193  83.706897  84.288089  84.013699
Hernandez High School     80.866860  80.660147  81.396140  80.857143
Holden High School        83.677165  83.324561  83.815534  84.698795
Huang High School         81.290284  81.512386  81.417476  80.305983
Johnson High School       81.260714  80.773431  80.616027  81.227564
Pena High School          83.807273  83.612000  84.335938  84.591160
Rodriguez High School     80.993127  80.629808  80.864811  80.376426
Shelton High School       84.122642  83.441964  84.373786  82.781671
Thomas High School        83.728850  84.254157  83.585542  83.831361
Wilson High School        83.939778  84.021452  83.764608  84.317673
Wright High School        83.833333  83.812757  84.156322  84.073171

In [18]:
# count the number of student that passed reading and reading for each school
reading_9gr = (student_data[student_data['grade'] == '9th']).groupby('school_name').mean()[['reading_score']]
reading_10gr = (student_data[student_data['grade'] == '10th']).groupby('school_name').mean()[['reading_score']]
reading_11gr = (student_data[student_data['grade'] == '11th']).groupby('school_name').mean()[['reading_score']]
reading_12gr = (student_data[student_data['grade'] == '12th']).groupby('school_name').mean()[['reading_score']]

# combine the lists and rename columns to clean up the look
reading_df = pd.merge(reading_9gr,reading_10gr, on='school_name')
reading_df  = reading_df .rename(columns={"reading_score_x":"9th","reading_score_y":"10th"})

reading_df = pd.merge(reading_df,reading_11gr, on='school_name')
reading_df = pd.merge(reading_df,reading_12gr, on='school_name')
reading_df  = reading_df .rename(columns={"reading_score_x":"11th","reading_score_y":"12th"})

reading_df

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564


,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [78]:
district_info_df['Per Student Budget'].median()

628.0

In [83]:
district_info_df.sort_values("Per Student Budget", ascending=True).reset_index()


,index,School Name,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Passing Overall
0,13,Wilson High School,Charter,2283,1319574,578.0,83.274201,83.989488,93.867718,96.539641,90.582567
1,6,Holden High School,Charter,427,248087,581.0,83.803279,83.814988,92.505855,96.252927,89.227166
2,1,Cabrera High School,Charter,1858,1081356,582.0,83.061895,83.975780,94.133477,97.039828,91.334769
3,14,Wright High School,Charter,1800,1049400,583.0,83.682222,83.955000,93.333333,96.611111,90.333333
4,11,Shelton High School,Charter,1761,1056600,600.0,83.359455,83.725724,93.867121,95.854628,89.892107
5,9,Pena High School,Charter,962,585858,609.0,83.839917,84.044699,94.594595,95.945946,90.540541
6,4,Griffin High School,Charter,1468,917500,625.0,83.351499,83.816757,93.392371,97.138965,90.599455
7,0,Bailey High School,District,4976,3124928,628.0,77.048432,81.033963,66.680064,81.933280,54.642283
8,10,Rodriguez High School,District,3999,2547363,637.0,76.842711,80.744686,66.366592,80.220055,52.988247
9,12,Thomas High School,Charter,1635,1043130,638.0,83.418349,83.848930,93.272171,97.308869,90.948012


In [85]:
# modified school_data_complete and create perstudent_budget
spending_ps_df = school_data_complete
spending_ps_df['Per Student Budget'] = (school_data_complete['budget'] / school_data_complete['size'])

# Create bins and bin labels for the Spending_PerStudent column
hp_bins = [550, 585, 630, 645, 675]
hp_labels = ["<$584", "$585-629", "$630-644","$645-675"]

# Bin the Spending_PerStudent column
# cut() returns a Pandas Series containing each of the binned column's values translated into their corresponding bins

spending_ps_df["Spending Ranges (Per Student)"] = pd.cut(spending_ps_df["Per Student Budget"], hp_bins, labels=hp_labels, include_lowest=True)
spending_ps_df.head(1)

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget,Per Student Budget,Spending Ranges (Per Student)
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635,655.0,$645-675


In [90]:
# get average score of math and reading for each bin
spend_df = spending_ps_df.groupby('Spending Ranges (Per Student)').mean()[['math_score','reading_score']]
spend_df 

,math_score,reading_score
Spending Ranges (Per Student),,
<$584,83.363065,83.964039
$585-629,79.982873,82.312643
$630-644,77.821056,81.301007
$645-675,77.049297,81.005604


In [92]:
# count the number of student that passed math and reading for each school
spend_df['Average Math Score']= (spending_ps_df[spending_ps_df['math_score'] >= 70]).groupby('Spending Ranges (Per Student)').count()[['math_score']]
spend_df['Average Reading Score'] = (spending_ps_df[spending_ps_df['reading_score'] >= 70]).groupby('Spending Ranges (Per Student)').count()[['reading_score']]
spend_df 

,math_score,reading_score,Average Math Score,Average Reading Score
Spending Ranges (Per Student),,,,
<$584,83.363065,83.964039,5967,6157
$585-629,79.982873,82.312643,7252,8114
$630-644,77.821056,81.301007,7996,9352
$645-675,77.049297,81.005604,8155,9987


In [19]:
# modified school_data_complete and create perstudent_budget
spending_ps_df = school_data_complete
spending_ps_df['Per Student Budget'] = (school_data_complete['budget'] / school_data_complete['size'])

# Create bins and bin labels for the Spending_PerStudent column
hp_bins = [550, 585, 630, 645, 675]
hp_labels = ["<$584", "$585-629", "$630-644","$645-675"]

# Bin the Spending_PerStudent column
# cut() returns a Pandas Series containing each of the binned column's values translated into their corresponding bins

spending_ps_df["Spending Ranges (Per Student)"] = pd.cut(spending_ps_df["Per Student Budget"], hp_bins, labels=hp_labels, include_lowest=True)

# get average score of math and reading for each bin
sum_df = spending_ps_df.groupby('Spending Ranges (Per Student)').mean()[['math_score','reading_score']]

# count the number of student that passed math and reading for each school
pct_math = (spending_ps_df[spending_ps_df['math_score'] >= 70]).groupby('Spending Ranges (Per Student)').count()[['math_score']]
pct_read = (spending_ps_df[spending_ps_df['reading_score'] >= 70]).groupby('Spending Ranges (Per Student)').count()[['reading_score']]

# count the number of student that passed reading and math
passed_both = spending_ps_df[(spending_ps_df['math_score'] >= 70) & (spending_ps_df['reading_score'] >= 70)]
pct_both = passed_both.groupby('Spending Ranges (Per Student)').count()[['student_name']]

# count the number of student for each bin
num_of_std = spending_ps_df.groupby('Spending Ranges (Per Student)').count()[['size']]

# combine the lists
sum_df = pd.merge(sum_df,pct_math, on='Spending Ranges (Per Student)')
sum_df = pd.merge(sum_df,pct_read, on='Spending Ranges (Per Student)')
sum_df = pd.merge(sum_df,pct_both, on='Spending Ranges (Per Student)')
sum_df = pd.merge(sum_df,num_of_std, on='Spending Ranges (Per Student)')

# Add and calculate columns for average
sum_df['% Passing Math'] = (100* sum_df['math_score_y'] / sum_df['size'])
sum_df['% Passing Reading'] = (100* sum_df['reading_score_y'] / sum_df['size'])
sum_df['% Overall Passing'] = (100* sum_df['student_name'] / sum_df['size'])

# Delete columns for numebr of students that passed math, reading, and both
sum_df = (sum_df.drop(['math_score_y', 'reading_score_y','student_name','size'], axis=1))

# Rename columns to clean up the look
sum_df = sum_df.rename(columns={"math_score_x":"Average Math Score","reading_score_x":"Average Reading Score"})
sum_df.round(2)

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.36,83.96,93.70,96.69,90.64
$585-629,79.98,82.31,79.11,88.51,70.94
$630-644,77.82,81.30,70.62,82.60,58.84
$645-675,77.05,81.01,66.23,81.11,53.53


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$584,83.46,83.93,93.46,96.61,90.37
$585-629,81.90,83.16,87.13,92.72,81.42
$630-644,78.52,81.62,73.48,84.39,62.86
$645-675,77.00,81.03,66.16,81.13,53.53


## Scores by School Size

* Perform the same operations as above, based on school size.

In [17]:
# modified school_data_complete and create perstudent_budget
school_size_df = school_data_complete

# Create bins and bin labels for the Spending_PerStudent column
hp_bins1 = [0, 1000, 2000, 5000]
hp_labels1 = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

# Bin the Spending_PerStudent column
# cut() returns a Pandas Series containing each of the binned column's values translated into their corresponding bins

school_size_df["School Size"] = pd.cut(school_size_df["size"], hp_bins1, labels=hp_labels1)

# get average score of math and reading for each bin
sum1_df = school_size_df.groupby('School Size').mean()[['math_score','reading_score']]

# count the number of student that passed math and reading for each school
pct_math1 = (school_size_df[school_size_df['math_score'] >= 70]).groupby('School Size').count()[['math_score']]
pct_read1 = (school_size_df[school_size_df['reading_score'] >= 70]).groupby('School Size').count()[['reading_score']]

# count the number of student that passed reading and math
passed_both = school_size_df[(school_size_df['math_score'] >= 70) & (school_size_df['reading_score'] >= 70)]
pct_both1 = passed_both.groupby('School Size').count()[['student_name']]

# count the number of student for each bin
num_of_std1 = school_size_df.groupby('School Size').count()[['size']]

# combine the lists
sum1_df = pd.merge(sum1_df,pct_math1, on='School Size')
sum1_df = pd.merge(sum1_df,pct_read1, on='School Size')
sum1_df = pd.merge(sum1_df,pct_both1, on='School Size')
sum1_df = pd.merge(sum1_df,num_of_std1, on='School Size')

# Add and calculate columns for average
sum1_df['% Passing Math'] = (100* sum1_df['math_score_y'] / sum1_df['size'])
sum1_df['% Passing Reading'] = (100* sum1_df['reading_score_y'] / sum1_df['size'])
sum1_df['% Overall Passing'] = (100* sum1_df['student_name'] / sum1_df['size'])

# Delete columns for numebr of students that passed math, reading, and both
sum1_df = (sum1_df.drop(['math_score_y', 'reading_score_y','student_name','size'], axis=1))

# Rename columns to clean up the look
sum1_df = sum1_df.rename(columns={"math_score_x":"Average Math Score","reading_score_x":"Average Reading Score"})

sum1_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.828654,83.974082,93.952484,96.040317,90.136789
Medium (1000-2000),83.372682,83.867989,93.616522,96.773058,90.624267
Large (2000-5000),77.477597,81.198674,68.652380,82.125158,56.574046


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

* Perform the same operations as above, based on school type

In [3]:
school_data_complete.head()

,Student ID,student_name,gender,grade,school_name,reading_score,math_score,School ID,type,size,budget
0,0,Paul Bradley,M,9th,Huang High School,66,79,0,District,2917,1910635
1,1,Victor Smith,M,12th,Huang High School,94,61,0,District,2917,1910635
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60,0,District,2917,1910635
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58,0,District,2917,1910635
4,4,Bonnie Ray,F,9th,Huang High School,97,84,0,District,2917,1910635


In [6]:
# modified school_data_complete and create perstudent_budget
school_type_df = school_data_complete
school_type_df = school_type_df.rename(columns={"type":"School Type"})

# get average score of math and reading for each school type
sum2_df = school_type_df.groupby('School Type').mean()[['math_score','reading_score']]

# count the number of student that passed math and reading for each school
pct_math2 = (school_type_df[school_type_df['math_score'] >= 70]).groupby('School Type').count()[['math_score']]
pct_read2 = (school_type_df[school_type_df['reading_score'] >= 70]).groupby('School Type').count()[['reading_score']]

# count the number of student that passed reading and math
passed_both = school_type_df[(school_type_df['math_score'] >= 70) & (school_type_df['reading_score'] >= 70)]
pct_both2 = passed_both.groupby('School Type').count()[['student_name']]

# count the number of student for each bin
num_of_std2 = school_type_df.groupby('School Type').count()[['size']]

# combine the lists
sum2_df = pd.merge(sum2_df,pct_math2, on='School Type')
sum2_df = pd.merge(sum2_df,pct_read2, on='School Type')
sum2_df = pd.merge(sum2_df,pct_both2, on='School Type')
sum2_df = pd.merge(sum2_df,num_of_std2, on='School Type')

# Add and calculate columns for average
sum2_df['% Passing Math'] = (100* sum2_df['math_score_y'] / sum2_df['size'])
sum2_df['% Passing Reading'] = (100* sum2_df['reading_score_y'] / sum2_df['size'])
sum2_df['% Overall Passing'] = (100* sum2_df['student_name'] / sum2_df['size'])

# Delete columns for numebr of students that passed math, reading, and both
sum2_df = (sum2_df.drop(['math_score_y', 'reading_score_y','student_name','size'], axis=1))

# Rename columns to clean up the look
sum2_df = sum2_df.rename(columns={"math_score_x":"Average Math Score","reading_score_x":"Average Reading Score"})

sum2_df

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.406183,83.902821,93.701821,96.645891,90.560932
District,76.987026,80.962485,66.518387,80.905249,53.695878


,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
